# Download Embeddings CELLxGENE 

> This module interfaces with the CZ CELLxGENE Discover Census package to download pre-computed embeddings for a subset of cells. These embeddings can be used as a reference for label transfer in single-cell analyses.


In [ ]:
#| default_exp Census

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from scanpy import AnnData

# Download Pre-Embeded Census Data 
This function `download_census_data` is designed to automate the process of downloading census data and retrieving embeddings for a specific organism and tissue type. It takes four parameters: `census_version` to specify the version of the census data, `embedding_uri` to provide the location of the embeddings data, `organism` to filter the data for a particular organism (like "homo_sapiens"), and `tissue` to specify the tissue type (like "blood"). The function returns an Anndata object that contains the embeddings corresponding to the filtered census data.


In [ ]:
#| default_exp census

In [ ]:
#| export


from scanpy import AnnData

def download_emb(
    census_version: str = '2023-12-15',  # The version date of the CELLxGENE census data to be used.
    embedding_uri: str = 's3://cellxgene-contrib-public/contrib/cell-census/soma/2023-12-15/CxG-contrib-2',  # The URI where the embeddings are stored, typically an S3 bucket path.
    organism: str = 'homo_sapiens',  # The scientific name of the organism for which the data is to be filtered.
    tissue: str = 'blood',  # The type of tissue for which the data is to be filtered
    test: bool = False  # Defines whether the data should be subsetted to specifically include macrophages to avoid downloading a very large dataset as the test case.
) -> AnnData:  # An Anndata object with the precomputed embeddings 
    """
    Download census data and retrieve embeddings for the specified organism and tissue with default parameters. 
    """
    import cellxgene_census
    from cellxgene_census.experimental import get_embedding

    # Open the census data for the given version
    census = cellxgene_census.open_soma(census_version=census_version)

    # Get the Anndata object for the specified organism and tissue
    if test:
        adata = cellxgene_census.get_anndata(
            census,
            organism=organism,
            measurement_name="RNA",
            obs_value_filter=f"tissue_general == '{tissue}' and cell_type == 'macrophage'"
        )
    else:
        adata = cellxgene_census.get_anndata(
            census,
            organism=organism,
            measurement_name="RNA",
            obs_value_filter=f"tissue_general == '{tissue}'"
        )

    # Retrieve embeddings using the soma_joinid from the Anndata object
    embeddings = get_embedding(census_version, embedding_uri, adata.obs["soma_joinid"].to_numpy())
    adata.obsm["emb"] = embeddings

    return adata



# Example Usage
The following code snippet demonstrates how to use the `download_and_embed_census_data` function to download and embed census data for human blood tissue. The function is called with default parameters for `census_version` and `embedding_uri`, while `organism` and `tissue` are set to 'homo_sapiens' and 'blood', respectively. To avoid downloading an extremely large dataset we will use the test case by setting the test lag to true, which is programmed to only download the macrophages. The result is stored in the variable `reference`, which contains the Anndata object with the embeddings.




In [ ]:
reference = download_census_data(organism='homo_sapiens', tissue='blood', test = True)

We can invoke the Anndata object to inspect its structure and understand the organization of the data it contains.



In [ ]:
reference

AnnData object with n_obs × n_vars = 7433 × 60664
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'
    obsm: 'emb'

We can access the embeddings generated by the universal cell embeddings model (UCE) which are stored in the 'emb' field of the .obsm attribute.
embeddings = reference.obsm['emb']



In [ ]:
reference.obsm['emb']

array([[-0.03808594, -0.00436401, -0.00092316, ...,  0.02258301,
         0.01843262, -0.01483154],
       [ 0.00213623, -0.00836182,  0.02612305, ...,  0.02392578,
        -0.00515747, -0.01940918],
       [-0.01513672, -0.00891113,  0.02539062, ...,  0.01312256,
         0.01538086, -0.01550293],
       ...,
       [-0.01538086, -0.01330566, -0.00454712, ..., -0.00640869,
        -0.00497437,  0.01556396],
       [-0.01025391,  0.03100586,  0.02111816, ..., -0.02893066,
         0.00787354,  0.06738281],
       [ 0.02087402, -0.0324707 ,  0.01879883, ..., -0.01397705,
        -0.0255127 ,  0.0213623 ]], dtype=float32)

Each cell downloaded from the CELLxGENE Discover Census package is represented by a 1280-dimensional vector, as observed in the output below when we check the shape of the embedding.


In [ ]:
reference.obsm['emb'].shape

(7433, 1280)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()